In [67]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import aot

In [68]:
# Load configuration files
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]

In [69]:
# AOT Test simplified parameters
total_video_number = core_settings['various']['total_video_number']  # Use total video number from original config
video_trials_per_run = 72  # Number of video trials per run
blank_trials_per_run = 12  # Number of blank trials per run
total_trials_per_run = video_trials_per_run + blank_trials_per_run

print(f"Total number of videos: {total_video_number}")
print(f"Video trials per run: {video_trials_per_run}")
print(f"Blank trials per run: {blank_trials_per_run}")
print(f"Total trials per run: {total_trials_per_run}")

Total number of videos: 2179
Video trials per run: 72
Blank trials per run: 12
Total trials per run: 84


In [70]:
# Generate list of all video names
all_video_names = []
for i in range(1, total_video_number + 1):
    video_name = str(i).zfill(4) + "_fw.mp4"  # Only use forward videos
    all_video_names.append(video_name)

print(f"Total forward videos: {len(all_video_names)}")
print(f"First few video names: {all_video_names[:10]}")

Total forward videos: 2179
First few video names: ['0001_fw.mp4', '0002_fw.mp4', '0003_fw.mp4', '0004_fw.mp4', '0005_fw.mp4', '0006_fw.mp4', '0007_fw.mp4', '0008_fw.mp4', '0009_fw.mp4', '0010_fw.mp4']


In [71]:
def generate_aottest_runs(subject_id="99"):
    """
    Generate two AOT test run configurations using the same 72 videos but in different orders
    
    Parameters:
    - subject_id: Subject ID (default 999 for testing)
    """
    
    # Randomly select 72 different forward videos (both runs use the same videos)
    selected_videos = random.sample(all_video_names, video_trials_per_run)
    
    def create_trial_list_with_blanks(videos):
        """Create trial list with blanks and shuffle order"""
        # Create trial list: 72 videos + 12 blanks
        trial_list = videos + ["blank"] * blank_trials_per_run
        
        # Shuffle trial order
        random.shuffle(trial_list)
        
        # Ensure no consecutive blanks (optional quality control)
        def avoid_consecutive_blanks(trial_list):
            trial_list = copy.deepcopy(trial_list)
            max_attempts = 100
            attempts = 0
            
            while attempts < max_attempts:
                consecutive_blanks = False
                for i in range(len(trial_list) - 1):
                    if trial_list[i] == "blank" and trial_list[i + 1] == "blank":
                        consecutive_blanks = True
                        break
                
                if not consecutive_blanks:
                    break
                    
                random.shuffle(trial_list)
                attempts += 1
                
            if attempts == max_attempts:
                print("Warning: Could not completely avoid consecutive blank trials")
                
            return trial_list
        
        return avoid_consecutive_blanks(trial_list)
    
    # Create trial lists with different orders for two runs
    trial_list_run1 = create_trial_list_with_blanks(selected_videos.copy())
    trial_list_run2 = create_trial_list_with_blanks(selected_videos.copy())
    
    # Create settings dictionaries
    settings_run1 = copy.deepcopy(stimuli_settings_temp)
    settings_run1["stimuli"]["movie_files"] = trial_list_run1
    
    settings_run2 = copy.deepcopy(stimuli_settings_temp)
    settings_run2["stimuli"]["movie_files"] = trial_list_run2
    
    # Generate filenames
    filename1 = f"experiment_settings_sub_{subject_id}_ses_aottest_run_01.yml"
    filename2 = f"experiment_settings_sub_{subject_id}_ses_aottest_run_02.yml"

    # Create save directory (if it doesn't exist)
    save_dir = settings_root_path / "aottest"
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Save files
    filepath1 = save_dir / filename1
    with open(filepath1, 'w') as outfile:
        yaml.dump(settings_run1, outfile, default_flow_style=False)

    filepath2 = save_dir / filename2
    with open(filepath2, 'w') as outfile:
        yaml.dump(settings_run2, outfile, default_flow_style=False)

    print(f"Generated config file: {filepath1}")
    print(f"Generated config file: {filepath2}")
    print(f"Both runs use the same {len(selected_videos)} videos, but in different orders")
    print(f"Total trials per run: {len(trial_list_run1)}")
    print(f"Video trials per run: {len([t for t in trial_list_run1 if t != 'blank'])}")
    print(f"Blank trials per run: {len([t for t in trial_list_run1 if t == 'blank'])}")
    
    return (settings_run1, settings_run2), (trial_list_run1, trial_list_run2), selected_videos

In [72]:
# Generate AOT test configuration - two runs using same videos but different orders
(settings_run1, settings_run2), (trial_list_run1, trial_list_run2), selected_videos = generate_aottest_runs(subject_id="99")

Generated config file: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_99_ses_aottest_run_01.yml
Generated config file: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_99_ses_aottest_run_02.yml
Both runs use the same 72 videos, but in different orders
Total trials per run: 84
Video trials per run: 72
Blank trials per run: 12


In [73]:
# Display selected video list
print("Selected 72 videos:")
for i, video in enumerate(selected_videos[:10]):
    print(f"{i+1:2d}: {video}")
print("...")
for i, video in enumerate(selected_videos[-5:], len(selected_videos)-4):
    print(f"{i:2d}: {video}")

print("\n" + "="*50)
print("Run 1 - First 20 trials:")
for i, trial in enumerate(trial_list_run1[:20]):
    print(f"{i+1:2d}: {trial}")
    
print(f"\n...\n")
print("Run 1 - Last 10 trials:")
for i, trial in enumerate(trial_list_run1[-10:], len(trial_list_run1)-9):
    print(f"{i:2d}: {trial}")

print("\n" + "="*50)
print("Run 2 - First 20 trials:")
for i, trial in enumerate(trial_list_run2[:20]):
    print(f"{i+1:2d}: {trial}")
    
print(f"\n...\n")
print("Run 2 - Last 10 trials:")
for i, trial in enumerate(trial_list_run2[-10:], len(trial_list_run2)-9):
    print(f"{i:2d}: {trial}")

Selected 72 videos:
 1: 0351_fw.mp4
 2: 1327_fw.mp4
 3: 0467_fw.mp4
 4: 1796_fw.mp4
 5: 0567_fw.mp4
 6: 0588_fw.mp4
 7: 0201_fw.mp4
 8: 2167_fw.mp4
 9: 1847_fw.mp4
10: 1012_fw.mp4
...
68: 1447_fw.mp4
69: 1604_fw.mp4
70: 1264_fw.mp4
71: 1376_fw.mp4
72: 0914_fw.mp4

Run 1 - First 20 trials:
 1: 0467_fw.mp4
 2: blank
 3: 1904_fw.mp4
 4: 2167_fw.mp4
 5: 1634_fw.mp4
 6: 1914_fw.mp4
 7: 2100_fw.mp4
 8: blank
 9: 0567_fw.mp4
10: 2028_fw.mp4
11: 0634_fw.mp4
12: 1376_fw.mp4
13: 0936_fw.mp4
14: 0337_fw.mp4
15: 1796_fw.mp4
16: 1090_fw.mp4
17: 0443_fw.mp4
18: 0459_fw.mp4
19: 1149_fw.mp4
20: 0818_fw.mp4

...

Run 1 - Last 10 trials:
75: 1110_fw.mp4
76: 1570_fw.mp4
77: 1220_fw.mp4
78: 1829_fw.mp4
79: 0914_fw.mp4
80: blank
81: 0928_fw.mp4
82: 1327_fw.mp4
83: 1562_fw.mp4
84: 0975_fw.mp4

Run 2 - First 20 trials:
 1: 1914_fw.mp4
 2: 2100_fw.mp4
 3: 0337_fw.mp4
 4: 1090_fw.mp4
 5: 1149_fw.mp4
 6: blank
 7: 1782_fw.mp4
 8: 1058_fw.mp4
 9: 0465_fw.mp4
10: 2077_fw.mp4
11: 0900_fw.mp4
12: 0553_fw.mp4
13: 13

In [74]:
# Verify results - check both runs
def verify_run(trial_list, run_name):
    video_count = len([t for t in trial_list if t != 'blank'])
    blank_count = len([t for t in trial_list if t == 'blank'])
    unique_videos = set([t for t in trial_list if t != 'blank'])
    
    print(f"{run_name}verification results:")
    print(f"✓ Total trials: {len(trial_list)} (expected: {total_trials_per_run})")
    print(f"✓ Video trials: {video_count} (expected: {video_trials_per_run})")
    print(f"✓ Blank trials: {blank_count} (expected: {blank_trials_per_run})")
    print(f"✓ Unique videos: {len(unique_videos)} (expected: {video_trials_per_run})")
    print(f"✓ All videos are forward (_fw): {all(v.endswith('_fw.mp4') for v in unique_videos)}")
    return unique_videos

print("Verification of two runs using same videos but different orders:")
print("="*60)

run1_videos = verify_run(trial_list_run1, "Run 1 ")
print()
run2_videos = verify_run(trial_list_run2, "Run 2 ")

print()
print("Video consistency check between runs:")
videos_match = run1_videos == run2_videos
print(f"✓ Both runs use exactly the same videos: {videos_match}")

if videos_match:
    print(f"✓ Number of shared videos: {len(run1_videos)}")
else:
    print(f"✗ Videos don't match!")
    print(f"  Run1 only: {run1_videos - run2_videos}")
    print(f"  Run2 only: {run2_videos - run1_videos}")

Verification of two runs using same videos but different orders:
Run 1 verification results:
✓ Total trials: 84 (expected: 84)
✓ Video trials: 72 (expected: 72)
✓ Blank trials: 12 (expected: 12)
✓ Unique videos: 72 (expected: 72)
✓ All videos are forward (_fw): True

Run 2 verification results:
✓ Total trials: 84 (expected: 84)
✓ Video trials: 72 (expected: 72)
✓ Blank trials: 12 (expected: 12)
✓ Unique videos: 72 (expected: 72)
✓ All videos are forward (_fw): True

Video consistency check between runs:
✓ Both runs use exactly the same videos: True
✓ Number of shared videos: 72


In [75]:
# Check for consecutive blanks in both runs
def check_consecutive_blanks(trial_list, run_name):
    consecutive_blanks = []
    for i in range(len(trial_list) - 1):
        if trial_list[i] == "blank" and trial_list[i + 1] == "blank":
            consecutive_blanks.append((i, i+1))

    if consecutive_blanks:
        print(f"Warning: {run_name}found {len(consecutive_blanks)} consecutive blanks:")
        for start, end in consecutive_blanks:
            print(f"  Position {start+1}-{end+1}")
    else:
        print(f"✓ {run_name}no consecutive blank trials")
    
    return len(consecutive_blanks)

print("Consecutive blank check:")
print("="*30)
run1_consecutive = check_consecutive_blanks(trial_list_run1, "Run 1 ")
run2_consecutive = check_consecutive_blanks(trial_list_run2, "Run 2 ")

print(f"\nSummary: Both runs avoid consecutive blanks: {run1_consecutive == 0 and run2_consecutive == 0}")

Consecutive blank check:
✓ Run 1 no consecutive blank trials
✓ Run 2 no consecutive blank trials

Summary: Both runs avoid consecutive blanks: True


In [76]:
# Generate configurations for subjects 1-9
print("Generating configurations for subjects 1-9:")
print("="*60)

for subject_num in range(1, 10):
    subject_id = f"{subject_num:02d}"  # Format as 001, 002, etc.
    print(f"\nGenerating for subject {subject_id}...")
    
    try:
        # Generate configurations for this subject
        (settings_run1, settings_run2), (trial_list_run1, trial_list_run2), selected_videos = generate_aottest_runs(subject_id=subject_id)
        
        # Quick verification
        run1_videos = set([t for t in trial_list_run1 if t != 'blank'])
        run2_videos = set([t for t in trial_list_run2 if t != 'blank'])
        videos_match = run1_videos == run2_videos
        
        print(f"  ✓ Subject {subject_id}: {len(selected_videos)} videos, consistency: {videos_match}")
        
    except Exception as e:
        print(f"  ✗ Error generating for subject {subject_id}: {e}")

print(f"\n{'='*60}")
print("Batch generation completed!")
print("Files saved in: aottest/ directory")

Generating configurations for subjects 1-9:

Generating for subject 01...
Generated config file: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_01_ses_aottest_run_01.yml
Generated config file: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_01_ses_aottest_run_02.yml
Both runs use the same 72 videos, but in different orders
Total trials per run: 84
Video trials per run: 72
Blank trials per run: 12
  ✓ Subject 01: 72 videos, consistency: True

Generating for subject 02...
Generated config file: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_02_ses_aottest_run_01.yml
Generated config file: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/expe

In [77]:
# Verify all generated configuration files
import os

aottest_dir = settings_root_path / "aottest"
print(f"Checking generated files in: {aottest_dir}")
print("="*60)

if aottest_dir.exists():
    generated_files = sorted(list(aottest_dir.glob("*.yml")))
    
    print(f"Total files generated: {len(generated_files)}")
    print("\nGenerated configuration files:")
    
    subjects_found = set()
    for file_path in generated_files:
        filename = file_path.name
        print(f"  {filename}")
        
        # Extract subject ID from filename
        if "sub_" in filename:
            sub_id = filename.split("sub_")[1][:3]
            subjects_found.add(sub_id)
    
    print(f"\nSubjects with configurations: {sorted(subjects_found)}")
    
    # Expected files check
    expected_subjects = [f"{i:03d}" for i in range(1, 10)]
    expected_files = []
    for sub_id in expected_subjects:
        expected_files.extend([
            f"experiment_settings_sub_{sub_id}_ses_aottest_run_01.yml",
            f"experiment_settings_sub_{sub_id}_ses_aottest_run_02.yml"
        ])
    
    print(f"\nExpected files: {len(expected_files)}")
    print(f"Generated files: {len(generated_files)}")
    print(f"All files generated successfully: {len(generated_files) == len(expected_files)}")
    
    # Check for missing files
    generated_names = {f.name for f in generated_files}
    missing_files = [f for f in expected_files if f not in generated_names]
    
    if missing_files:
        print(f"\nMissing files: {missing_files}")
    else:
        print("\n✓ All expected configuration files have been generated successfully!")
        
else:
    print(f"Directory {aottest_dir} does not exist!")

Checking generated files in: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest
Total files generated: 20

Generated configuration files:
  experiment_settings_sub_01_ses_aottest_run_01.yml
  experiment_settings_sub_01_ses_aottest_run_02.yml
  experiment_settings_sub_02_ses_aottest_run_01.yml
  experiment_settings_sub_02_ses_aottest_run_02.yml
  experiment_settings_sub_03_ses_aottest_run_01.yml
  experiment_settings_sub_03_ses_aottest_run_02.yml
  experiment_settings_sub_04_ses_aottest_run_01.yml
  experiment_settings_sub_04_ses_aottest_run_02.yml
  experiment_settings_sub_05_ses_aottest_run_01.yml
  experiment_settings_sub_05_ses_aottest_run_02.yml
  experiment_settings_sub_06_ses_aottest_run_01.yml
  experiment_settings_sub_06_ses_aottest_run_02.yml
  experiment_settings_sub_07_ses_aottest_run_01.yml
  experiment_settings_sub_07_ses_aottest_run_02.yml
  experiment_settings_sub_08_ses_aottest_run_01.yml
  experiment_se